# View and export review of long recordings from single wearer
Review and export excerpt data and images of long single participant recordings using the equivital vest and downloaded with the Qiosk interface. 

In [1]:
import sys
import os
import time
import datetime as dt
import math
import numpy as np 
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import heartpy as hp

from scipy.signal import butter,filtfilt
from scipy import interpolate
from scipy.interpolate import interp1d

In [2]:
%load_ext autoreload
%autoreload 1
%aimport qex

In [19]:
%reload_ext autoreload

## Find csvs and do a quick review and export

In [20]:
# location of the files when exported by QIOSK
path = '../SoloConcerting/SoloRecordings'

df_datafiles = qex.qiosk_recordings(path,'Present','/')
df_datafiles

TypeError: qiosk_recordings() takes 2 positional arguments but 3 were given

In [17]:
dataLoc = df_datafiles.loc[1,'FullLoc']

In [18]:
sep = '/'
matched = qex.matched_files(dataLoc,path,sep) # outputs locations of csv 
for eq_file_loc in matched:
    print(eq_file_loc)
    if not eq_file_loc.lower().endswith('.sem'):
        V = pd.read_csv(eq_file_loc,skipinitialspace=True)
        print(V.columns)
        qex.test_plot_signals(V)

../SoloConcerting/SoloRecordings/CSV/ECG-Pilot_1-3420755-23020100.csv


IndexError: list index out of range